In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/games-dataset/games.csv


In [2]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148498 sha256=2e029f405f28174d7ec3ff6c0564a65de0dc1d66b0df0420da0ed57ce75c6c40
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [3]:
import pandas as pd
import chess
import chess.svg
import random

In [4]:
df = pd.read_csv('/kaggle/input/games-dataset/games.csv')

In [5]:
df.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [6]:
df.shape

(20058, 16)

In [7]:
df_copy = df.copy()

In [8]:
df_copy

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [9]:
df_copy = df_copy[df_copy['turns']<40]

In [10]:
df_copy

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
5,MsoDV9wj,False,1.504240e+12,1.504240e+12,5,draw,draw,10+0,trelynn17,1250,franklin14532,1002,e4 c5 Nf3 Qa5 a3,B27,Sicilian Defense: Mongoose Variation,4
6,qwU9rasv,True,1.504230e+12,1.504230e+12,33,resign,white,10+0,capa_jr,1520,daniel_likes_chess,1423,d4 d5 e4 dxe4 Nc3 Nf6 f3 exf3 Nxf3 Nc6 Bb5 a6 ...,D00,Blackmar-Diemer Gambit: Pietrowsky Defense,10
7,RVN0N3VK,False,1.503680e+12,1.503680e+12,9,resign,black,15+30,daniel_likes_chess,1413,soultego,2108,e4 Nc6 d4 e5 d5 Nce7 c3 Ng6 b4,B00,Nimzowitsch Defense: Kennedy Variation | Link...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20048,dnexZDsv,True,1.499869e+12,1.499870e+12,25,resign,white,10+10,mateuslichess,1252,jamboger,1233,e4 e6 d4 d5 exd5 Qxd5 Nc3 Bb4 a3 Bxc3+ bxc3 Ne...,C01,French Defense: Exchange Variation,5
20050,nYOvevdh,True,1.499814e+12,1.499814e+12,9,outoftime,white,10+0,jamboger,1243,yamaguchipolgar,1142,c4 e5 d4 exd4 Qxd4 Nf6 Bg5 Be7 e4,A20,English Opening: King's English Variation,2
20052,EopEqqAa,True,1.499812e+12,1.499812e+12,37,resign,white,10+10,jamboger,1219,samael88,1250,c4 e6 d4 b6 Nc3 Bb7 Nf3 g6 h4 Bg7 Bg5 f6 Bf4 d...,A40,English Defense,4
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2


In [11]:
df_copy.shape

(5675, 16)

In [12]:
df_copy.isnull().sum()

id                0
rated             0
created_at        0
last_move_at      0
turns             0
victory_status    0
winner            0
increment_code    0
white_id          0
white_rating      0
black_id          0
black_rating      0
moves             0
opening_eco       0
opening_name      0
opening_ply       0
dtype: int64

In [13]:
import chess

def evaluate_board_advanced(board, white_rating, black_rating, victory_status):
    eval_map = {chess.PAWN: 1,chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9}

    score = 0
    for piece in eval_map:
        score+= len(board.pieces(piece, chess.WHITE)) * eval_map[piece]
        score-= len(board.pieces(piece, chess.BLACK)) * eval_map[piece]

    rating_diff = white_rating - black_rating
    score+= rating_diff * 0.01

    if victory_status == 'white':
        score+=10
    elif victory_status == 'black':
        score-=10
    
    return score
        

In [14]:
def alpha_beta_advanced(board, depth, alpha, beta, is_maximizing, white_rating, black_rating, victory_status):
    if depth == 0 or board.is_game_over():
        return evaluate_board_advanced(board, white_rating, black_rating, victory_status)
        
    if is_maximizing:
        max_eval = float('-inf')
        for move in board.legal_moves:
            board.push(move)
            eval = alpha_beta_advanced(board, depth - 1, alpha, beta, False,
                                       white_rating, black_rating, victory_status)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval = alpha_beta_advanced(board, depth - 1, alpha, beta, True,
                                       white_rating, black_rating, victory_status)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def get_best_move_advanced(board, depth, white_rating, black_rating, victory_status):
    best_move = None
    best_value = float('-inf') if board.turn == chess.WHITE else float('inf')
    
    for move in board.legal_moves:
        board.push(move)
        board_value = alpha_beta_advanced(board, depth - 1, float('-inf'), float('inf'),
                                          not board.turn, white_rating, black_rating, victory_status)
        board.pop()
        
        if (board.turn == chess.WHITE and board_value > best_value) or (board.turn == chess.BLACK and board_value < best_value):
            best_value = board_value
            best_move = move
            
    return best_move

In [15]:
import chess
import random

# Sample game setup using additional features from dataset
sample_game = df.sample(1).iloc[0]
white_rating = sample_game['white_rating']
black_rating = sample_game['black_rating']
victory_status = sample_game['winner']

# Define the board display function (as before or adapt if needed)
def display_board(board):
    print(board)  # Simple text display of the board

# Run a game simulation
def play_advanced_game():
    board = chess.Board()
    while not board.is_game_over():
        display_board(board)  # Display the board state after each move

        if board.turn == chess.WHITE:
            # Player input
            player_move = input("Enter your move in algebraic notation (e.g., e2e4 or Nf3): ")
            try:
                # Try pushing move in SAN format
                board.push_san(player_move)
            except ValueError:
                print("Invalid move, please enter in proper SAN format (e.g., e2e4, Nf3).")
                continue
        else:
            # AI's move
            ai_move = get_best_move_advanced(board, depth=2, white_rating=white_rating, black_rating=black_rating, victory_status=victory_status)
            board.push(ai_move)

    # Display final result
    display_board(board)  # Show the final board state
    if board.is_checkmate():
        print("Checkmate! Winner:", "White" if board.turn == chess.BLACK else "Black")
    elif board.is_stalemate() or board.is_insufficient_material():
        print("Draw!")

# Start the game
play_advanced_game()


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


Enter your move in algebraic notation (e.g., e2e4 or Nf3):  bye


Invalid move, please enter in proper SAN format (e.g., e2e4, Nf3).
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


KeyboardInterrupt: Interrupted by user

In [16]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 56.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.0 MB/s eta 0:00:00:00:0100:01


In [17]:
!npm install localtunnel

m##################] | reify:yargs: http fetch GET 200 https://registry.npmjs.o/regis
added 22 packages in 3s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.5.0 -> 10.9.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.9.0
npm notice Run npm install -g npm@10.9.0 to update!
npm notice 


In [32]:
%%writefile app.py

import streamlit as st
import chess
import chess.svg
import random

def chess_game():
    # Initialize a new chess board only once, using session state to persist the board state
    if 'board' not in st.session_state:
        st.session_state.board = chess.Board()

    # Display the chess board
    st.subheader("Chess Game")
    st.image(chess.svg.board(st.session_state.board), use_column_width=True)

    # Create an input field for the player's move
    move = st.text_input("Enter your move (e.g., e2e4):")

    if move:
        try:
            # Apply the player's move if it is valid
            st.session_state.board.push_san(move)
            st.success(f"Your move {move} was successful!")

            # Check if the game is over after the player's move
            if st.session_state.board.is_checkmate():
                st.success("Checkmate! You win!")
            elif st.session_state.board.is_stalemate():
                st.success("Stalemate! It's a draw.")
            elif st.session_state.board.is_insufficient_material():
                st.success("Draw due to insufficient material.")
            else:
                # Generate a random legal move for the AI
                legal_moves = list(st.session_state.board.legal_moves)
                if legal_moves:
                    ai_move = random.choice(legal_moves)
                    st.session_state.board.push(ai_move)
                    st.success(f"AI move: {st.session_state.board.san(ai_move)}")
                    
        except ValueError:
            st.error("Invalid move! Please enter a move in proper SAN format.")
        except AssertionError as e:
            st.error(f"Move error: {e}")

    # Display the updated chessboard
    st.subheader("Updated Chess Board:")
    st.image(chess.svg.board(st.session_state.board), use_column_width=True)

if __name__ == "__main__":
    st.title("Chess Game with Basic AI")
    chess_game()



Overwriting app.py


In [33]:
!curl ipv4.icanhazip.com

35.229.169.226


In [34]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://vast-bottles-sing.loca.lt
^C
